## Get a filtration from Gudhi

In [1]:
import gudhi as gd
import tadasets
sphere = tadasets.dsphere(n=50, d=2, r=1, noise=0, seed=42)
alpha = gd.AlphaComplex(points=sphere)
st = alpha.create_simplex_tree()
filtration = list(st.get_filtration())

## Convert it to boundary map data

In [2]:
from persistent_laplacians.filtrations import simplices_by_dimension, compute_boundary_matrices, filtration_hash_map

unique_filtration_values = sorted(list(set([f for (_, f) in filtration])))
simplices_by_dim, simplices_by_dim_only_filt = simplices_by_dimension(filtration)
boundary_matrices = compute_boundary_matrices(simplices_by_dim)
boundary_maps_index_dict = filtration_hash_map(filtration, simplices_by_dim_only_filt)

## Compute homology using persistent laplacians
For now, compute only homology (using SVD)

In [3]:
from persistent_laplacians import persistent_laplacians as pl
result = pl.process_tda(
    boundary_matrices,
    boundary_maps_index_dict
)

## Barcodes 

In [4]:
from persistent_laplacians.barcodes import compute_barcodes
print({k: v for k,v in result[1].items() if v > 0})
barcodes = compute_barcodes(result, unique_filtration_values)
print(barcodes)
barcodes = [
    (q, (unique_filtration_values[i], unique_filtration_values[j]))
    for q in barcodes.keys()
    for (i, j) in barcodes[q].keys()
]
sorted(barcodes)

{(124, 150): 3, (118, 180): 2, (96, 165): 1, (84, 152): 1, (88, 113): 2, (95, 181): 1, (100, 118): 1, (109, 150): 1, (153, 156): 6, (95, 172): 1, (139, 155): 4, (126, 132): 4, (143, 177): 2, (87, 182): 1, (147, 154): 4, (88, 178): 1, (95, 183): 1, (104, 181): 1, (144, 168): 2, (97, 144): 1, (80, 104): 1, (92, 174): 1, (128, 162): 3, (137, 142): 4, (110, 141): 1, (125, 172): 2, (77, 101): 1, (161, 174): 2, (96, 150): 1, (105, 146): 1, (171, 174): 2, (105, 176): 1, (92, 186): 1, (108, 183): 1, (115, 139): 2, (67, 74): 1, (96, 187): 1, (126, 186): 1, (97, 153): 1, (123, 171): 2, (119, 166): 2, (117, 159): 2, (117, 120): 5, (96, 162): 1, (109, 132): 1, (125, 170): 2, (104, 146): 1, (67, 108): 1, (150, 171): 2, (106, 166): 1, (161, 182): 1, (146, 149): 4, (104, 174): 1, (112, 160): 2, (127, 154): 3, (155, 167): 2, (71, 83): 2, (89, 92): 4, (90, 143): 1, (100, 144): 1, (114, 123): 3, (112, 144): 2, (99, 184): 1, (83, 133): 1, (155, 161): 4, (96, 146): 1, (126, 149): 3, (97, 181): 1, (75, 79)

[(1, (0.020421464162303817, 0.02775126521477527)),
 (1, (0.022738317284251776, 0.025602542421691465)),
 (1, (0.02546175064832349, 0.027562930776185896)),
 (1, (0.02906069519472558, 0.0292751263572475)),
 (1, (0.03760807066406452, 0.04030672465663575)),
 (1, (0.04250752904013125, 0.04293962587045624)),
 (1, (0.044913703617217894, 0.04495798491737818)),
 (1, (0.05429917177781154, 0.05611354104356754)),
 (1, (0.059899315127668544, 0.1058157960883063)),
 (1, (0.06100763850766194, 0.08832296550232062)),
 (1, (0.0694517097337741, 0.07052809999108911)),
 (1, (0.07629993344229474, 0.0763039217778961)),
 (1, (0.07867295640614468, 0.08458221447589057)),
 (1, (0.07984334183687994, 0.3445383368670274)),
 (1, (0.08204308018320211, 0.09634960452032912)),
 (1, (0.08535724083291552, 0.08536093285726777)),
 (1, (0.08552386563819075, 0.0855339522610657)),
 (1, (0.0863462139447869, 0.08824266716822335)),
 (1, (0.08694565715396549, 0.0869456580426308)),
 (1, (0.08843082116528883, 0.09153208796329397)),
 (

## Verify with Gudhi

In [5]:
gudhi_persistence = [(q, (i, j)) for (q, (i,j)) in st.persistence() if q > 0]
print(sorted(gudhi_persistence))
sorted(gudhi_persistence) == sorted(barcodes)

[(1, (0.020421464162303817, 0.02775126521477527)), (1, (0.022738317284251776, 0.025602542421691465)), (1, (0.02546175064832349, 0.027562930776185896)), (1, (0.02906069519472558, 0.0292751263572475)), (1, (0.03760807066406452, 0.04030672465663575)), (1, (0.04250752904013125, 0.04293962587045624)), (1, (0.044913703617217894, 0.04495798491737818)), (1, (0.05429917177781154, 0.05611354104356754)), (1, (0.059899315127668544, 0.1058157960883063)), (1, (0.06100763850766194, 0.08832296550232062)), (1, (0.0694517097337741, 0.07052809999108911)), (1, (0.07629993344229474, 0.0763039217778961)), (1, (0.07867295640614468, 0.08458221447589057)), (1, (0.07984334183687994, 0.3445383368670274)), (1, (0.08204308018320211, 0.09634960452032912)), (1, (0.08535724083291552, 0.08536093285726777)), (1, (0.08552386563819075, 0.0855339522610657)), (1, (0.0863462139447869, 0.08824266716822335)), (1, (0.08694565715396549, 0.0869456580426308)), (1, (0.08843082116528883, 0.09153208796329397)), (1, (0.09115194518810

True